# Pandas API on Spark

## Przygotowania

Na początku pobierz dane, które będziemy przetwarzali 

In [ ]:
%%sh
wget https://jankiewicz.pl/bigdata/bigdata-sp/netflix.zip

Następnie je rozpakuj 

In [ ]:
%%sh
unzip netflix.zip

Załaduj je do systemu plików HDFS

In [ ]:
%%sh
hadoop fs -put movie_titles.csv /tmp/
hadoop fs -put rates /tmp/

Sprawdź czy są dostępne w ich źródłowych (dla naszego przetwarzania) miejscach 

In [ ]:
%%sh
hadoop fs -ls /tmp

Podglądnij jak wygląda ich budowa 

In [ ]:
%%sh
hadoop fs -head /tmp/movie_titles.csv

In [ ]:
%%sh
hadoop fs -head /tmp/rates/part-00.csv

## Odczyt danych

Na początek kontekst

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/06 20:54:10 INFO SparkEnv: Registering MapOutputTracker
23/12/06 20:54:10 INFO SparkEnv: Registering BlockManagerMaster
23/12/06 20:54:11 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
23/12/06 20:54:11 INFO SparkEnv: Registering OutputCommitCoordinator


Ustawmy stosowne zmienne środowiskowe, a następnie zaimportujmy potrzebne biblioteki

In [4]:
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"

In [5]:
import pyspark.pandas as ps

Zaczytajmy dane do *Pandas Dataframe on Spark*

Na początku informacje na temat filmów

In [7]:
movies_ps = ps.read_csv('/tmp/movie_titles.csv')

Dowiedzmy się jak wygląda "budowa" naszych danych

In [8]:
movies_ps.shape

(17770, 3)

In [9]:
movies_ps.columns

In [13]:
movies_ps.head()

,ID,Year,Title
0,1,2003,Dinosaur Planet
1,2,2004,Isle of Man TT 2004 Review
2,3,1997,Character
3,4,1994,Paula Abdul's Get Up & Dance
4,5,2004,The Rise and Fall of ECW


Czas zatem na oceny

In [10]:
rates_ps = ps.read_csv('/tmp/rates')

Dowiedzmy się jak wygląda "budowa" naszych danych

In [11]:
rates_ps.shape

(14780972, 4)

In [12]:
rates_ps.columns

In [14]:
rates_ps.head()

,date,film_id,user_id,rate
0,2003-04-07,1256,406841,4
1,2003-04-07,1256,79527,4
2,2003-04-07,1256,108859,4
3,2003-04-07,1260,1816643,3
4,2003-04-07,1260,1827071,3


To co, jesteśmy w domu? Zatem do dzieła. Trzy proste zadania

# Zadania

## Zadanie 1

Określ ile filmów nie uzyskało żadnej oceny. Utwórz *Pandas Dataframe on Spark*, który będzie zawierał komplet informacji filmów ograniczonych tylko do tych, które mają chociaż jedną ocenę.  

In [ ]:
movies_rates_ps = ps.merge(movies_ps, rates_ps, left_on='ID', right_on='film_id')

In [ ]:
movies_ps['ID'].nunique()

In [ ]:
movies_rates_ps['ID'].nunique()

In [ ]:
movies_rates_uniques_ps = movies_rates_ps.drop_duplicates(subset=['ID'])
movies_rates_uniques_ps.shape

## Zadanie 2

Dla każdej dekady premiery filmów utwórz dwa rankingi

- trzech filmów o największej liczbie ocen
- trzech filmów o największej średniej ocen

In [ ]:
movies_rates_ps['decade'] = (movies_rates_ps['Year'] // 10) * 10

In [ ]:
movies_rates_ps.groupby(['decade', 'ID']).agg({'rate': 'count'}).sort_values(by=['decade', 'rate'], ascending=[True, False]).groupby('decade').head(3).reset_index()

In [ ]:
movies_rates_ps.groupby(['decade', 'ID']).agg({'rate': 'mean'}).sort_values(by=['decade', 'rate'], ascending=[True, False]).groupby('decade').head(3).reset_index()

## Zadanie 3

Dla trzech wybranych użytkowników, wśród tych, którzy ocenili co najemniej 10 filmów, wyznacz trzy najbardziej rekomendowane filmy do obejrzenia. 

Wskaż źródła opisujące zastosowane przez Ciebie podejście. 

Koniecznie uzupełnij kod szczegółowymi komentarzami. 